In [28]:
swiss_file = "swiss_tournament_20251028_164548.pkl"  # R5
swiss_file = "swiss_tournament_20251029_165045.pkl"  # R200

# Swiss Tournament Analysis

This notebook analyzes the results from a Swiss-system tournament between checkpoints.

In [29]:
import sys
from pathlib import Path

sys.path.append(str(Path("../..").resolve()))
import os

if os.path.basename(os.getcwd()) == "E02_win_rate":
    os.chdir("../..")

In [30]:
import pickle
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import numpy as np

## Load Tournament Data

In [31]:
# Load the tournament results
with open(swiss_file, "rb") as f:
    tournament_data = pickle.load(f)

print("Tournament data loaded successfully!")
print(f"\nAvailable keys: {tournament_data.keys()}")

Tournament data loaded successfully!

Available keys: dict_keys(['config', 'final_standings', 'match_results', 'timestamp'])


## Tournament Configuration

In [32]:
config = tournament_data["config"]
print(f"Number of Rounds: {config['num_rounds']}")
print(f"Double Swiss: {config['double_swiss']}")
print(f"McMahon Scoring: {config['mcmahon']}")
print(f"Checkpoint Folder: {config['folder']}")
print(f"Timestamp: {tournament_data['timestamp']}")

Number of Rounds: 200
Double Swiss: True
McMahon Scoring: True
Checkpoint Folder: CHECKPOINTS//E02_win_rate//
Timestamp: 20251029_165045


## Final Standings

In [33]:
# Convert standings to DataFrame
standings = tournament_data["final_standings"]
standings_df = pd.DataFrame(standings)

# Display top 20
print(f"\nTotal participants: {len(standings_df)}")
print(f"\nTop 20 Final Standings:")
top_20 = standings_df.head(20)[
    ["name", "score", "wins", "draws", "losses", "matches_played", "epoch"]
]
print(top_20.to_string(index=False))

standings_df.head()


Total participants: 1043

Top 20 Final Standings:
                                 name  score  wins  draws  losses  matches_played  epoch
20251023_1326-E02_win_rate_epoch_0031  233.0   233      0     167             400     31
20251023_1326-E02_win_rate_epoch_0028  232.0   231      2     167             400     28
20251023_1325-E02_win_rate_epoch_0021  228.0   228      0     172             400     21
20251023_1325-E02_win_rate_epoch_0022  227.5   227      1     172             400     22
20251023_1326-E02_win_rate_epoch_0025  227.0   227      0     173             400     25
20251023_1337-E02_win_rate_epoch_0187  225.0   224      2     174             400    187
20251023_1325-E02_win_rate_epoch_0018  225.0   224      2     174             400     18
20251023_1325-E02_win_rate_epoch_0020  224.5   223      3     174             400     20
20251023_1326-E02_win_rate_epoch_0029  224.0   224      0     176             400     29
20251023_1326-E02_win_rate_epoch_0024  224.0   224      0  

,name,path,epoch,score,wins,draws,losses,matches_played
0,20251023_1326-E02_win_rate_epoch_0031,CHECKPOINTS\E02_win_rate\20251023_1326-E02_win...,31,233.0,233,0,167,400
1,20251023_1326-E02_win_rate_epoch_0028,CHECKPOINTS\E02_win_rate\20251023_1326-E02_win...,28,232.0,231,2,167,400
2,20251023_1325-E02_win_rate_epoch_0021,CHECKPOINTS\E02_win_rate\20251023_1325-E02_win...,21,228.0,228,0,172,400
3,20251023_1325-E02_win_rate_epoch_0022,CHECKPOINTS\E02_win_rate\20251023_1325-E02_win...,22,227.5,227,1,172,400
4,20251023_1326-E02_win_rate_epoch_0025,CHECKPOINTS\E02_win_rate\20251023_1326-E02_win...,25,227.0,227,0,173,400


## Score Distribution

In [34]:
fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x=standings_df["score"],
        nbinsx=30,
        name="Score Distribution",
        marker_color="blue",
    )
)

fig.update_layout(
    title="Score Distribution Across All Participants",
    xaxis_title="Total Score",
    yaxis_title="Number of Bots",
    height=400,
)
fig.show()

## Epoch vs Performance

In [35]:
# Scatter plot: Epoch vs Score
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=standings_df["epoch"],
        y=standings_df["score"],
        mode="markers",
        marker=dict(
            size=8,
            color=standings_df["score"],
            colorscale="Viridis",
            showscale=True,
            colorbar=dict(title="Score"),
        ),
        text=standings_df["name"].str[-15:],  # Show last 15 chars of name
        hovertemplate="<b>%{text}</b><br>Epoch: %{x}<br>Score: %{y}<extra></extra>",
    )
)

fig.update_layout(
    title="Epoch vs Tournament Score",
    xaxis_title="Training Epoch",
    yaxis_title="Tournament Score",
    height=500,
    xaxis=dict(range=[0, 100]),
)
fig.show()

In [36]:
# Win rate vs epoch
standings_df["win_rate"] = standings_df["wins"] / standings_df["matches_played"]

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=standings_df["epoch"],
        y=standings_df["win_rate"],
        mode="markers",
        marker=dict(
            size=8,
            color=standings_df["win_rate"],
            colorscale="RdYlGn",
            showscale=True,
            colorbar=dict(title="Win Rate"),
        ),
        text=standings_df["name"].str[-15:],
        hovertemplate="<b>%{text}</b><br>Epoch: %{x}<br>Win Rate: %{y:.3f}<extra></extra>",
    )
)

fig.update_layout(
    title="Win Rate vs Training Epoch",
    xaxis_title="Training Epoch",
    yaxis_title="Win Rate",
    height=500,
    # xaxis=dict(range=[150, 300]),
)
fig.show()

In [37]:
from scipy import stats

# Calculate correlation between win rate and score

corr_winrate_score, p_winrate_score = stats.pearsonr(
    standings_df["win_rate"], standings_df["score"]
)

print("=" * 60)
print("WIN RATE vs SCORE CORRELATION ANALYSIS")
print("=" * 60)
print(f"Correlation coefficient: {corr_winrate_score:.4f}")
print(f"P-value: {p_winrate_score:.4e}")
print(f"Statistically significant: {'Yes' if p_winrate_score < 0.05 else 'No'}")
print("=" * 60)

# Create scatter plot
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=standings_df["win_rate"],
        y=standings_df["score"],
        mode="markers",
        marker=dict(
            size=6,
            color=standings_df["epoch"],
            colorscale="Viridis",
            showscale=True,
            colorbar=dict(title="Epoch"),
        ),
        text=standings_df["name"].str[-15:],
        hovertemplate="<b>%{text}</b><br>Win Rate: %{x:.3f}<br>Score: %{y}<extra></extra>",
    )
)

fig.update_layout(
    title=f"Win Rate vs Score (correlation: {corr_winrate_score:.4f})",
    xaxis_title="Win Rate",
    yaxis_title="Tournament Score",
    height=500,
)
fig.show()

WIN RATE vs SCORE CORRELATION ANALYSIS
Correlation coefficient: 0.9977
P-value: 0.0000e+00
Statistically significant: Yes


## Match Results Analysis

In [38]:
# Convert match results to DataFrame
matches = tournament_data["match_results"]
matches_df = pd.DataFrame(matches)

print(f"Total matches played: {len(matches_df)}")
print(f"\nMatch results columns: {matches_df.columns.tolist()}")
matches_df.head(10)

Total matches played: 208400

Match results columns: ['round', 'match_id', 'player1', 'player2', 'player1_epoch', 'player2_epoch', 'p1_wins', 'p2_wins', 'draws']


,round,match_id,player1,player2,player1_epoch,player2_epoch,p1_wins,p2_wins,draws
0,1,R1_M1_G1,20251023_1439-E02_win_rate_epoch_1014,20251023_1439-E02_win_rate_epoch_1013,1014,1013,1,0,0
1,1,R1_M1_G2,20251023_1439-E02_win_rate_epoch_1013,20251023_1439-E02_win_rate_epoch_1014,1013,1014,0,1,0
2,1,R1_M2_G1,20251023_1438-E02_win_rate_epoch_1012,20251023_1438-E02_win_rate_epoch_1011,1012,1011,1,0,0
3,1,R1_M2_G2,20251023_1438-E02_win_rate_epoch_1011,20251023_1438-E02_win_rate_epoch_1012,1011,1012,0,1,0
4,1,R1_M3_G1,20251023_1438-E02_win_rate_epoch_1010,20251023_1438-E02_win_rate_epoch_1009,1010,1009,0,1,0
5,1,R1_M3_G2,20251023_1438-E02_win_rate_epoch_1009,20251023_1438-E02_win_rate_epoch_1010,1009,1010,0,1,0
6,1,R1_M4_G1,20251023_1438-E02_win_rate_epoch_1008,20251023_1438-E02_win_rate_epoch_1007,1008,1007,1,0,0
7,1,R1_M4_G2,20251023_1438-E02_win_rate_epoch_1007,20251023_1438-E02_win_rate_epoch_1008,1007,1008,1,0,0
8,1,R1_M5_G1,20251023_1438-E02_win_rate_epoch_1006,20251023_1438-E02_win_rate_epoch_1005,1006,1005,1,0,0
9,1,R1_M5_G2,20251023_1438-E02_win_rate_epoch_1005,20251023_1438-E02_win_rate_epoch_1006,1005,1006,1,0,0


In [39]:
# Analyze match outcomes
total_p1_wins = matches_df["p1_wins"].sum()
total_p2_wins = matches_df["p2_wins"].sum()
total_draws = matches_df["draws"].sum()
total_games = total_p1_wins + total_p2_wins + total_draws

print("=" * 60)
print("OVERALL MATCH STATISTICS")
print("=" * 60)
print(f"Player 1 Wins: {total_p1_wins} ({total_p1_wins/total_games*100:.1f}%)")
print(f"Player 2 Wins: {total_p2_wins} ({total_p2_wins/total_games*100:.1f}%)")
print(f"Draws: {total_draws} ({total_draws/total_games*100:.1f}%)")
print(f"Total Games: {total_games}")
print("=" * 60)

OVERALL MATCH STATISTICS
Player 1 Wins: 103941 (49.9%)
Player 2 Wins: 103851 (49.8%)
Draws: 608 (0.3%)
Total Games: 208400


In [40]:
# Match outcomes pie chart
fig = go.Figure(
    data=[
        go.Pie(
            labels=["Player 1 Wins", "Player 2 Wins", "Draws"],
            values=[total_p1_wins, total_p2_wins, total_draws],
            marker_colors=["#1f77b4", "#ff7f0e", "#2ca02c"],
        )
    ]
)

fig.update_layout(title="Overall Match Outcomes Distribution", height=400)
fig.show()

## Performance by Round

In [41]:
# Analyze performance by round
round_stats = matches_df.groupby("round").agg(
    {
        "p1_wins": "sum",
        "p2_wins": "sum",
        "draws": "sum",
    }
)

round_stats["total_games"] = (
    round_stats["p1_wins"] + round_stats["p2_wins"] + round_stats["draws"]
)
round_stats["p1_win_rate"] = round_stats["p1_wins"] / round_stats["total_games"]
round_stats["draw_rate"] = round_stats["draws"] / round_stats["total_games"]

fig = make_subplots(rows=2, cols=1, subplot_titles=("Games per Round", "Outcome Rates"))

# Total games per round
fig.add_trace(
    go.Scatter(
        x=round_stats.index,
        y=round_stats["total_games"],
        mode="lines+markers",
        name="Total Games",
    ),
    row=1,
    col=1,
)

# Outcome rates
fig.add_trace(
    go.Scatter(
        x=round_stats.index,
        y=round_stats["p1_win_rate"],
        mode="lines+markers",
        name="P1 Win Rate",
    ),
    row=2,
    col=1,
)

# fig.add_trace(
#     go.Scatter(
#         x=round_stats.index,
#         y=round_stats["draw_rate"],
#         mode="lines+markers",
#         name="Draw Rate",
#     ),
#     row=2,
#     col=1,
# )

fig.update_xaxes(title_text="Round", row=2, col=1)
fig.update_yaxes(title_text="Games", row=1, col=1)
fig.update_yaxes(title_text="Rate", row=2, col=1)
fig.update_layout(height=700, title_text="Round-by-Round Analysis")
fig.show()

## Top Performers Analysis

In [42]:
# Top 10 performers detailed stats
top_10 = standings_df.head(10).copy()
top_10["win_rate"] = top_10["wins"] / top_10["matches_played"]
top_10["draw_rate"] = top_10["draws"] / top_10["matches_played"]
top_10["loss_rate"] = top_10["losses"] / top_10["matches_played"]

fig = go.Figure()

# Stacked bar chart
fig.add_trace(
    go.Bar(
        name="Wins",
        x=top_10["name"].str[-15:],
        y=top_10["win_rate"],
        marker_color="green",
    )
)

fig.add_trace(
    go.Bar(
        name="Draws",
        x=top_10["name"].str[-15:],
        y=top_10["draw_rate"],
        marker_color="yellow",
    )
)

fig.add_trace(
    go.Bar(
        name="Losses",
        x=top_10["name"].str[-15:],
        y=top_10["loss_rate"],
        marker_color="red",
    )
)

fig.update_layout(
    barmode="stack",
    title="Top 10 Performers - Match Outcome Distribution",
    xaxis_title="Bot Name (last 15 chars)",
    yaxis_title="Rate",
    height=500,
)
fig.show()

## Top 10 Ranking Progression Across Rounds

Track how the rankings of the final top 10 bots evolved throughout the tournament.

In [ ]:
# Get top 10 final bots
top_10_names = standings_df.head(10)["name"].tolist()

# Get all bot names
bot_names = standings_df["name"].tolist()

# Initialize scores for all bots across rounds
num_rounds = config["num_rounds"]
bot_scores_by_round = {name: [0.0] * (num_rounds + 1) for name in bot_names}

# Process matches round by round to calculate cumulative scores
for round_num in range(1, num_rounds + 1):
    round_matches = matches_df[matches_df["round"] == round_num]

    for _, match in round_matches.iterrows():
        p1 = match["player1"]
        p2 = match["player2"]

        # Calculate points (1 for win, 0.5 for draw, 0 for loss)
        p1_points = match["p1_wins"] + 0.5 * match["draws"]
        p2_points = match["p2_wins"] + 0.5 * match["draws"]

        # Add to current round (cumulative)
        bot_scores_by_round[p1][round_num] = (
            bot_scores_by_round[p1][round_num - 1] + p1_points
        )
        bot_scores_by_round[p2][round_num] = (
            bot_scores_by_round[p2][round_num - 1] + p2_points
        )

    # Copy forward scores for bots that didn't play this round
    for bot in bot_names:
        # Check if bot had any update this round
        played_this_round = any(
            bot in [match["player1"], match["player2"]]
            for _, match in round_matches.iterrows()
        )

        if not played_this_round:
            bot_scores_by_round[bot][round_num] = bot_scores_by_round[bot][
                round_num - 1
            ]

# Calculate rankings for each round
rankings_by_round = {name: [] for name in top_10_names}

for round_num in range(num_rounds + 1):
    # Get scores for this round
    round_scores = [(name, bot_scores_by_round[name][round_num]) for name in bot_names]
    # Sort by score descending
    round_scores.sort(key=lambda x: x[1], reverse=True)
    # Create rank mapping
    rank_map = {name: rank + 1 for rank, (name, _) in enumerate(round_scores)}

    # Store ranks for top 10 bots
    for name in top_10_names:
        rankings_by_round[name].append(rank_map[name])

print(f"Tracking rankings across {num_rounds} rounds for top 10 final bots")

Tracking rankings across 200 rounds for top 10 final bots



In [76]:
# Create ranking progression plot
fig = go.Figure()

rounds = list(range(num_rounds + 1))

# Add a line for each top 10 bot
for i, bot_name in enumerate(top_10_names):
    final_rank = i + 1
    fig.add_trace(
        go.Scatter(
            x=rounds,
            y=rankings_by_round[bot_name],
            mode="lines+markers",
            name=f"#{final_rank}: {bot_name[-20:]}",
            line=dict(width=2),
            marker=dict(size=6),
            hovertemplate=f"<b>{bot_name[-30:]}</b><br>Round: %{{x}}<br>Rank: %{{y}}<extra></extra>",
        )
    )

fig.update_layout(
    title="Ranking Progression of Top 10 Final Bots Across Tournament Rounds",
    xaxis_title="Round",
    yaxis_title="Rank",
    height=600,
    # xaxis=dict(range=[0, 10]),
    # hovermode="x unified",
    # legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99),
)

# Reverse y-axis so rank 1 is at top
fig.update_yaxes(autorange="reversed")

fig.show()

In [68]:
# Calculate ranking stability metric
# For each round, calculate how many of the final top 10 were in top 10 at that round

stability_by_round = []
for round_num in range(num_rounds + 1):
    # Get scores for this round
    round_scores = [(name, bot_scores_by_round[name][round_num]) for name in bot_names]
    round_scores.sort(key=lambda x: x[1], reverse=True)
    top_10_this_round = {name for name, _ in round_scores[:10]}

    # Count overlap with final top 10
    overlap = len(top_10_this_round.intersection(set(top_10_names)))
    stability_by_round.append(overlap)

# Plot stability
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=rounds,
        y=stability_by_round,
        mode="lines+markers",
        fill="tozeroy",
        name="Top 10 Stability",
        line=dict(color="blue", width=3),
        marker=dict(size=8),
        hovertemplate="Round: %{x}<br>Final Top 10 in Current Top 10: %{y}/10<extra></extra>",
    )
)

fig.add_hline(
    y=10,
    line_dash="dash",
    line_color="green",
    annotation_text="Perfect Stability (10/10)",
    annotation_position="right",
)

fig.update_layout(
    title="Ranking Stability: How Many Final Top 10 Bots Are in Top 10 Each Round",
    xaxis_title="Round",
    yaxis_title="Number of Final Top 10 in Current Top 10",
    height=400,
    yaxis=dict(range=[0, 10.5]),
)

fig.show()

# Find when rankings stabilize (e.g., when 8+ of final top 10 are in top 10)
stability_threshold = 8
stable_round = next(
    (r for r, s in enumerate(stability_by_round) if s >= stability_threshold), None
)

print("=" * 60)
print("RANKING STABILITY ANALYSIS")
print("=" * 60)
if stable_round is not None:
    print(
        f"Rankings stabilized at round {stable_round} ({stability_threshold}/10 threshold)"
    )
else:
    print(f"Rankings never fully stabilized to {stability_threshold}/10 threshold")
print(f"Final round stability: {stability_by_round[-1]}/10")
print("=" * 60)

RANKING STABILITY ANALYSIS
Rankings stabilized at round 0 (8/10 threshold)
Final round stability: 3/10


## Epoch Correlation Analysis

In [43]:
from scipy import stats

# Calculate correlation between epoch and performance metrics
corr_score, p_score = stats.pearsonr(standings_df["epoch"], standings_df["score"])
corr_winrate, p_winrate = stats.pearsonr(
    standings_df["epoch"], standings_df["win_rate"]
)

print("=" * 60)
print("EPOCH CORRELATION ANALYSIS")
print("=" * 60)
print(f"Epoch vs Score:")
print(f"  Correlation: {corr_score:.4f}")
print(f"  P-value: {p_score:.4e}")
print(f"  Significant: {'Yes' if p_score < 0.05 else 'No'}")
print()
print(f"Epoch vs Win Rate:")
print(f"  Correlation: {corr_winrate:.4f}")
print(f"  P-value: {p_winrate:.4e}")
print(f"  Significant: {'Yes' if p_winrate < 0.05 else 'No'}")
print("=" * 60)

EPOCH CORRELATION ANALYSIS
Epoch vs Score:
  Correlation: -0.0585
  P-value: 5.8950e-02
  Significant: No

Epoch vs Win Rate:
  Correlation: -0.0567
  P-value: 6.7376e-02
  Significant: No


In [44]:
summary = {
    "Total Bots": len(standings_df),
    "Total Rounds": config["num_rounds"],
    "Total Matches": len(matches_df),
    "Total Games": total_games,
    "Avg Matches per Bot": standings_df["matches_played"].mean(),
    "Avg Score": standings_df["score"].mean(),
    "Median Score": standings_df["score"].median(),
    "Avg Win Rate": standings_df["win_rate"].mean(),
    "Top Score": standings_df["score"].max(),
    "Top Bot": standings_df.iloc[0]["name"][-30:],
    "Top Epoch": standings_df.iloc[0]["epoch"],
}

print("=" * 60)
print("TOURNAMENT SUMMARY")
print("=" * 60)
for key, value in summary.items():
    if isinstance(value, float):
        print(f"{key:.<40} {value:.3f}")
    else:
        print(f"{key:.<40} {value}")
print("=" * 60)

TOURNAMENT SUMMARY
Total Bots.............................. 1043
Total Rounds............................ 200
Total Matches........................... 208400
Total Games............................. 208400
Avg Matches per Bot..................... 399.616
Avg Score............................... 200.000
Median Score............................ 200.000
Avg Win Rate............................ 0.498
Top Score............................... 233.000
Top Bot................................. 3_1326-E02_win_rate_epoch_0031
Top Epoch............................... 31


## Bradley-Terry Scoring

Calculate Bradley-Terry scores to estimate relative strength of each bot based on match results.

In [45]:
from utils.metrics.bradley_terry import calculate_BradleyTerry

# Build win matrix from match results
# W[i,j] = number of wins of bot i over bot j
bot_names = standings_df["name"].tolist()
n_bots = len(bot_names)

# Create name to index mapping
name_to_idx = {name: idx for idx, name in enumerate(bot_names)}

# Initialize win matrix as DataFrame
W = pd.DataFrame(0.0, index=bot_names, columns=bot_names)

# Populate win matrix from match results
for _, match in matches_df.iterrows():
    p1_name = match["player1"]
    p2_name = match["player2"]

    # Add wins to matrix
    W.loc[p1_name, p2_name] += match["p1_wins"]
    W.loc[p2_name, p1_name] += match["p2_wins"]

    # Handle draws (split equally)
    if match["draws"] > 0:
        W.loc[p1_name, p2_name] += match["draws"] * 0.5
        W.loc[p2_name, p1_name] += match["draws"] * 0.5

print(f"Win matrix constructed: {W.shape}")
print(
    f"Total games recorded: {W.sum().sum() / 2:.0f}"
)  # Divide by 2 since matrix counts each game twice

Win matrix constructed: (1043, 1043)
Total games recorded: 104200

Total games recorded: 104200


In [46]:
# Initialize scores (uniform initialization)
initial_scores = {name: 1.0 for name in bot_names}

# Calculate Bradley-Terry scores
bt_scores = calculate_BradleyTerry(
    score=initial_scores,
    W=W,
    EPOCHS=100,
    diff_threshold=1e-4,
    normalize=True,
    verbose=True,
)

print("\nBradley-Terry calculation complete!")

Calculating Bradley-Terry for 1043 agents.



BT Epoch: 100%|██████████| 1043/1043 [00:14<00:00, 73.87it/s]


Diff norm2: 0.003908
['1.341', '1.436', '1.347', '1.292', '1.398', '1.236', '1.461', '1.266', '1.343', '1.290', '1.346', '1.207', '1.291', '1.299', '1.253', '1.232', '1.257', '1.219', '1.288', '1.213', '1.222', '1.206', '1.258', '1.239', '1.172', '1.200', '1.197', '1.267', '1.234', '1.214', '1.239', '1.310', '1.179', '1.267', '1.194', '1.214', '1.156', '1.194', '1.208', '1.241', '1.213', '1.154', '1.186', '1.258', '1.161', '1.223', '1.213', '1.168', '1.147', '1.163', '1.227', '1.253', '1.152', '1.206', '1.167', '1.172', '1.386', '1.180', '1.151', '1.186', '1.167', '1.135', '1.228', '1.167', '1.133', '1.160', '1.148', '1.160', '1.230', '1.259', '1.201', '1.144', '1.136', '1.153', '1.118', '1.113', '1.270', '1.166', '1.259', '1.132', '1.111', '1.124', '1.126', '1.175', '1.240', '1.296', '1.213', '1.156', '1.121', '1.144', '1.111', '1.154', '1.107', '1.108', '1.107', '1.221', '1.129', '1.122', '1.164', '1.104', '1.190', '1.153', '1.125', '1.134', '1.133', '1.171', '1.141', '1.205', '1.113

BT Epoch: 100%|██████████| 1043/1043 [00:14<00:00, 71.49it/s] 


Diff norm2: 0.002549
['1.833', '1.853', '1.728', '1.570', '1.819', '1.418', '1.860', '1.502', '1.667', '1.620', '1.640', '1.371', '1.541', '1.605', '1.523', '1.459', '1.516', '1.403', '1.541', '1.394', '1.390', '1.323', '1.480', '1.410', '1.296', '1.352', '1.368', '1.488', '1.404', '1.369', '1.393', '1.572', '1.347', '1.487', '1.304', '1.333', '1.237', '1.326', '1.342', '1.418', '1.415', '1.261', '1.246', '1.509', '1.285', '1.402', '1.391', '1.313', '1.225', '1.297', '1.394', '1.460', '1.248', '1.371', '1.337', '1.278', '1.680', '1.344', '1.291', '1.376', '1.315', '1.224', '1.384', '1.294', '1.196', '1.279', '1.240', '1.286', '1.436', '1.420', '1.398', '1.294', '1.230', '1.238', '1.179', '1.133', '1.431', '1.292', '1.433', '1.225', '1.118', '1.204', '1.184', '1.329', '1.411', '1.483', '1.355', '1.275', '1.179', '1.275', '1.135', '1.240', '1.177', '1.174', '1.113', '1.367', '1.211', '1.180', '1.313', '1.196', '1.361', '1.225', '1.217', '1.227', '1.251', '1.283', '1.242', '1.373', '1.203

...

Diff norm2: 0.000619
['2.841', '2.649', '2.441', '2.018', '2.570', '1.667', '2.563', '1.848', '2.204', '2.144', '2.112', '1.577', '1.906', '2.078', '1.912', '1.792', '1.909', '1.647', '1.902', '1.643', '1.604', '1.471', '1.775', '1.652', '1.455', '1.544', '1.576', '1.793', '1.621', '1.578', '1.586', '1.952', '1.555', '1.802', '1.435', '1.473', '1.319', '1.492', '1.511', '1.657', '1.686', '1.375', '1.306', '1.868', '1.432', '1.652', '1.627', '1.494', '1.284', '1.451', '1.617', '1.734', '1.341', '1.582', '1.565', '1.408', '2.111', '1.553', '1.456', '1.617', '1.505', '1.319', '1.590', '1.428', '1.246', '1.402', '1.337', '1.434', '1.707', '1.629', '1.666', '1.470', '1.327', '1.318', '1.216', '1.131', '1.655', '1.446', '1.661', '1.336', '1.111', '1.278', '1.224', '1.520', '1.622', '1.733', '1.531', '1.413', '1.233', '1.434', '1.136', '1.328', '1.244', '1.216', '1.079', '1.556', '1.308', '1.246', '1.495', '1.282', '1.581', '1.306', '1.320', '1.330', '1.395', '1.410', '1.345', '1.585', '1.284

BT Epoch: 100%|██████████| 1043/1043 [00:10<00:00, 102.24it/s]


Diff norm2: 0.000363
['2.994', '2.762', '2.537', '2.073', '2.672', '1.693', '2.654', '1.888', '2.270', '2.207', '2.168', '1.596', '1.947', '2.133', '1.956', '1.828', '1.953', '1.672', '1.941', '1.668', '1.625', '1.486', '1.807', '1.678', '1.468', '1.562', '1.596', '1.826', '1.642', '1.599', '1.607', '1.994', '1.574', '1.835', '1.446', '1.489', '1.325', '1.508', '1.527', '1.680', '1.714', '1.384', '1.312', '1.907', '1.444', '1.677', '1.650', '1.510', '1.287', '1.464', '1.640', '1.762', '1.349', '1.602', '1.586', '1.419', '2.161', '1.572', '1.469', '1.640', '1.522', '1.326', '1.610', '1.439', '1.248', '1.411', '1.345', '1.447', '1.735', '1.651', '1.693', '1.485', '1.334', '1.324', '1.217', '1.130', '1.680', '1.460', '1.684', '1.345', '1.110', '1.282', '1.225', '1.538', '1.644', '1.759', '1.547', '1.424', '1.236', '1.448', '1.134', '1.334', '1.248', '1.218', '1.075', '1.574', '1.316', '1.252', '1.510', '1.287', '1.602', '1.313', '1.328', '1.338', '1.407', '1.422', '1.352', '1.605', '1.290

BT Epoch:  90%|█████████ | 942/1043 [00:09<00:00, 101.65it/s]


KeyboardInterrupt: 

## Bradley-Terry Rankings

In [50]:
# Create Bradley-Terry rankings DataFrame
bt_df = pd.DataFrame(
    [
        {
            "name": name,
            "bt_score": score,
            "epoch": standings_df[standings_df["name"] == name]["epoch"].iloc[0],
        }
        for name, score in bt_scores.items()
    ]
)

bt_df = bt_df.sort_values("bt_score", ascending=False).reset_index(drop=True)
bt_df["bt_rank"] = range(1, len(bt_df) + 1)

# Merge with original standings
standings_with_bt = standings_df.merge(
    bt_df[["name", "bt_score", "bt_rank"]], on="name", how="left"
)

# Compare Swiss rank vs BT rank
standings_with_bt["rank_diff"] = (
    standings_with_bt.index + 1 - standings_with_bt["bt_rank"]
)

print("Top 20 by Bradley-Terry Score:")
print(
    standings_with_bt.head(20)[
        ["name", "score", "bt_score", "bt_rank", "rank_diff", "epoch"]
    ].to_string(index=False)
)

standings_with_bt.head()

Top 20 by Bradley-Terry Score:
                                 name  score  bt_score  bt_rank  rank_diff  epoch
20251023_1326-E02_win_rate_epoch_0031  233.0  3.089141        1          0     31
20251023_1326-E02_win_rate_epoch_0028  232.0  2.832367        2          0     28
20251023_1325-E02_win_rate_epoch_0021  228.0  2.594493        5         -2     21
20251023_1325-E02_win_rate_epoch_0022  227.5  2.106472       11         -7     22
20251023_1326-E02_win_rate_epoch_0025  227.0  2.733692        3          2     25
20251023_1337-E02_win_rate_epoch_0187  225.0  1.707432       29        -23    187
20251023_1325-E02_win_rate_epoch_0018  225.0  2.709620        4          3     18
20251023_1325-E02_win_rate_epoch_0020  224.5  1.911662       18        -10     20
20251023_1326-E02_win_rate_epoch_0029  224.0  2.309123        6          3     29
20251023_1326-E02_win_rate_epoch_0024  224.0  2.244659        7          3     24
20251023_1325-E02_win_rate_epoch_0023  223.5  2.201103        8    

,name,path,epoch,score,wins,draws,losses,matches_played,win_rate,bt_score,bt_rank,rank_diff
0,20251023_1326-E02_win_rate_epoch_0031,CHECKPOINTS\E02_win_rate\20251023_1326-E02_win...,31,233.0,233,0,167,400,0.5825,3.089141,1,0
1,20251023_1326-E02_win_rate_epoch_0028,CHECKPOINTS\E02_win_rate\20251023_1326-E02_win...,28,232.0,231,2,167,400,0.5775,2.832367,2,0
2,20251023_1325-E02_win_rate_epoch_0021,CHECKPOINTS\E02_win_rate\20251023_1325-E02_win...,21,228.0,228,0,172,400,0.5700,2.594493,5,-2
3,20251023_1325-E02_win_rate_epoch_0022,CHECKPOINTS\E02_win_rate\20251023_1325-E02_win...,22,227.5,227,1,172,400,0.5675,2.106472,11,-7
4,20251023_1326-E02_win_rate_epoch_0025,CHECKPOINTS\E02_win_rate\20251023_1326-E02_win...,25,227.0,227,0,173,400,0.5675,2.733692,3,2


In [51]:
# Bradley-Terry score distribution
fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x=bt_df["bt_score"],
        nbinsx=30,
        name="BT Score Distribution",
        marker_color="purple",
    )
)

fig.update_layout(
    title="Bradley-Terry Score Distribution",
    xaxis_title="Bradley-Terry Score",
    yaxis_title="Number of Bots",
    height=400,
)
fig.show()

## Bradley-Terry vs Swiss Score Comparison

In [62]:
# Scatter plot: Swiss score vs BT score
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=standings_with_bt["score"],
        y=standings_with_bt["bt_score"],
        mode="markers",
        marker=dict(
            size=8,
            color=standings_with_bt["epoch"],
            colorscale="Viridis",
            showscale=True,
            colorbar=dict(title="Epoch"),
            opacity=0.8,
        ),
        text=standings_with_bt["name"].str[-15:],
        hovertemplate="<b>%{text}</b><br>Swiss: %{x:.2f}<br>BT: %{y:.4f}<br>Epoch: %{marker.color}<extra></extra>",
    )
)

# # Add diagonal line (perfect correlation)
# min_val = 0
# max_swiss = standings_with_bt["score"].max()
# max_bt = standings_with_bt["bt_score"].max()

# fig.add_trace(
#     go.Scatter(
#         x=[0, max_swiss],
#         y=[0, max_bt * (max_swiss / max_swiss)],
#         mode="lines",
#         line=dict(dash="dash", color="gray"),
#         name="Reference",
#         showlegend=False,
#     )
# )

fig.update_layout(
    title="Swiss Score vs Bradley-Terry Score",
    xaxis_title="Swiss Tournament Score",
    yaxis_title="Bradley-Terry Score",
    height=500,
)
fig.show()

In [63]:
# Correlation analysis
from scipy.stats import pearsonr, spearmanr

pearson_corr, pearson_p = pearsonr(
    standings_with_bt["score"], standings_with_bt["bt_score"]
)
spearman_corr, spearman_p = spearmanr(
    standings_with_bt["score"], standings_with_bt["bt_score"]
)

print("=" * 60)
print("CORRELATION ANALYSIS: Swiss Score vs Bradley-Terry Score")
print("=" * 60)
print(f"Pearson Correlation: {pearson_corr:.4f} (p={pearson_p:.4e})")
print(f"Spearman Correlation: {spearman_corr:.4f} (p={spearman_p:.4e})")
print("=" * 60)

CORRELATION ANALYSIS: Swiss Score vs Bradley-Terry Score
Pearson Correlation: 0.8983 (p=0.0000e+00)
Spearman Correlation: 0.9297 (p=0.0000e+00)

CORRELATION ANALYSIS: Swiss Score vs Bradley-Terry Score
Pearson Correlation: 0.8983 (p=0.0000e+00)
Spearman Correlation: 0.9297 (p=0.0000e+00)


## Bradley-Terry Score vs Epoch

In [64]:
# BT score vs epoch
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=standings_with_bt["epoch"],
        y=standings_with_bt["bt_score"],
        mode="markers",
        marker=dict(
            size=8,
            color=standings_with_bt["bt_score"],
            colorscale="RdYlGn",
            showscale=True,
            colorbar=dict(title="BT Score"),
            opacity=0.8,
        ),
        text=standings_with_bt["name"].str[-15:],
        hovertemplate="<b>%{text}</b><br>Epoch: %{x}<br>BT Score: %{y:.4f}<extra></extra>",
    )
)

fig.update_layout(
    title="Bradley-Terry Score vs Training Epoch",
    xaxis_title="Training Epoch",
    yaxis_title="Bradley-Terry Score",
    height=500,
)
fig.show()

In [55]:
# Epoch correlation with BT score
bt_epoch_corr, bt_epoch_p = pearsonr(
    standings_with_bt["epoch"], standings_with_bt["bt_score"]
)

print("=" * 60)
print("EPOCH CORRELATION WITH BRADLEY-TERRY")
print("=" * 60)
print(f"Correlation: {bt_epoch_corr:.4f}")
print(f"P-value: {bt_epoch_p:.4e}")
print(f"Significant: {'Yes' if bt_epoch_p < 0.05 else 'No'}")
print("=" * 60)

EPOCH CORRELATION WITH BRADLEY-TERRY
Correlation: -0.1142
P-value: 2.2005e-04
Significant: Yes

EPOCH CORRELATION WITH BRADLEY-TERRY
Correlation: -0.1142
P-value: 2.2005e-04
Significant: Yes


## Ranking Disagreements

Identify bots where Swiss ranking and Bradley-Terry ranking differ significantly.

In [56]:
# Find largest ranking disagreements
standings_with_bt["swiss_rank"] = standings_with_bt.index + 1
standings_with_bt["abs_rank_diff"] = abs(standings_with_bt["rank_diff"])

disagreements = standings_with_bt.nlargest(10, "abs_rank_diff")[
    ["name", "swiss_rank", "bt_rank", "rank_diff", "score", "bt_score", "epoch"]
]

print("Top 10 Ranking Disagreements:")
print(disagreements.to_string(index=False))

Top 10 Ranking Disagreements:
                                 name  swiss_rank  bt_rank  rank_diff  score  bt_score  epoch
20251023_1426-E02_win_rate_epoch_0889         420      876       -456  202.5  0.753695    889
20251023_1332-E02_win_rate_epoch_0119         209      614       -405  208.5  0.941345    119
20251023_1332-E02_win_rate_epoch_0116         809      418        391  192.5  1.070335    116
20251023_1432-E02_win_rate_epoch_0950         619      242        377  197.5  1.220178    950
20251023_1355-E02_win_rate_epoch_0439         815      439        376  192.0  1.054056    439
20251023_1427-E02_win_rate_epoch_0899         488      860       -372  200.5  0.769304    899
20251023_1315-E02_win_rate_epoch_0002         914      545        369  188.5  0.983143      2
20251023_1401-E02_win_rate_epoch_0527         496      851       -355  200.5  0.774509    527
20251023_1414-E02_win_rate_epoch_0707         823      486        337  192.0  1.023727    707
20251023_1403-E02_win_rate_epo

In [57]:
# Visualize ranking differences
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=standings_with_bt["swiss_rank"],
        y=standings_with_bt["bt_rank"],
        mode="markers",
        marker=dict(
            size=8,
            color=standings_with_bt["abs_rank_diff"],
            colorscale="Reds",
            showscale=True,
            colorbar=dict(title="Rank Difference"),
        ),
        text=standings_with_bt["name"].str[-15:],
        hovertemplate="<b>%{text}</b><br>Swiss Rank: %{x}<br>BT Rank: %{y}<br>Diff: %{marker.color}<extra></extra>",
    )
)

# Add diagonal (perfect agreement)
max_rank = len(standings_with_bt)
fig.add_trace(
    go.Scatter(
        x=[1, max_rank],
        y=[1, max_rank],
        mode="lines",
        line=dict(dash="dash", color="gray"),
        name="Perfect Agreement",
        showlegend=False,
    )
)

fig.update_layout(
    title="Swiss Rank vs Bradley-Terry Rank",
    xaxis_title="Swiss Rank",
    yaxis_title="Bradley-Terry Rank",
    height=500,
)
fig.update_xaxes(autorange="reversed")
fig.update_yaxes(autorange="reversed")
fig.show()

## Bradley-Terry Summary

Compare the top performers by both ranking systems.

In [58]:
# Side-by-side comparison of top 10
print("=" * 100)
print("TOP 10 COMPARISON: Swiss vs Bradley-Terry")
print("=" * 100)

swiss_top10 = standings_with_bt.head(10)[
    ["name", "score", "bt_score", "bt_rank"]
].copy()
swiss_top10.insert(0, "Swiss_Rank", range(1, 11))

bt_top10 = (
    standings_with_bt.sort_values("bt_rank")
    .head(10)[["name", "bt_score", "score"]]
    .copy()
)
bt_top10.insert(0, "BT_Rank", range(1, 11))
bt_top10["swiss_rank"] = bt_top10["name"].map(
    dict(zip(standings_with_bt["name"], standings_with_bt.index + 1))
)

print("\nTop 10 by Swiss Score:")
print(swiss_top10.to_string(index=False))

print("\n" + "-" * 100)
print("\nTop 10 by Bradley-Terry Score:")
print(bt_top10.to_string(index=False))
print("=" * 100)

TOP 10 COMPARISON: Swiss vs Bradley-Terry

Top 10 by Swiss Score:
 Swiss_Rank                                  name  score  bt_score  bt_rank
          1 20251023_1326-E02_win_rate_epoch_0031  233.0  3.089141        1
          2 20251023_1326-E02_win_rate_epoch_0028  232.0  2.832367        2
          3 20251023_1325-E02_win_rate_epoch_0021  228.0  2.594493        5
          4 20251023_1325-E02_win_rate_epoch_0022  227.5  2.106472       11
          5 20251023_1326-E02_win_rate_epoch_0025  227.0  2.733692        3
          6 20251023_1337-E02_win_rate_epoch_0187  225.0  1.707432       29
          7 20251023_1325-E02_win_rate_epoch_0018  225.0  2.709620        4
          8 20251023_1325-E02_win_rate_epoch_0020  224.5  1.911662       18
          9 20251023_1326-E02_win_rate_epoch_0029  224.0  2.309123        6
         10 20251023_1326-E02_win_rate_epoch_0024  224.0  2.244659        7

-------------------------------------------------------------------------------------------------